# Inferir
#### En esta lección, inferirá opiniones y temas a partir de reseñas de productos y artículos de noticias.



In [1]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [2]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [3]:
## Texto de revisión del producto

revision_lampara = """Necesitaba una linda lámpara para mi dormitorio, y esta tenía \
almacenamiento adicional y un precio no demasiado alto. \
Lo tengo rápido. La cuerda de nuestra lámpara se rompió durante el \
viaje y la compañía felizmente envió uno nuevo. \
Llegó a los pocos días también. Fue fácil de poner \
juntos. Me faltaba una pieza, así que contacté a su \
servicio de apoyo y muy rápidamente me consiguieron la pieza que faltaba! \
Lumina me parece una gran empresa que se preocupa \
sobre sus clientes y productos!!"""



## Sentiment (positive/negative)

In [4]:
prompt = f"""¿Cuál es el sentimiento de la siguiente revisión del producto,
que está delimitado con triple backticks?

Dé su respuesta en una sola palabra, ya sea "positiva" \
o "negativo".

Texto de la reseña: '''{revision_lampara}'''
"""
response = get_completion(prompt)
print(response)


Positiva.


## Identificar tipos de emociones

In [5]:
prompt = f"""Identifique una lista de emociones que el escritor de la \
siguiente revisión es expresa. No incluya más de \
cinco elementos de la lista. Formatee su respuesta como una lista de \
palabras en minúsculas separadas por comas.
Texto de la reseña: '''{revision_lampara}'''
"""
response = get_completion(prompt)
print(response)


contento, satisfecho, agradecido, impresionado, confiado


## Identificar la ira



In [6]:
prompt = f"""¿El escritor de la siguiente reseña está expresando enojo?\
La reseña está delimitada con triples comillas invertidas. \
Dé su respuesta como sí o no.
Texto de la reseña: '''{revision_lampara}'''
"""
response = get_completion(prompt)
print(response)


No.


## Extraiga el nombre del producto y de la empresa de las reseñas de los clientes

In [7]:
prompt = f"""Identifique los siguientes elementos del texto de revisión:
- Artículo comprado por el revisor
- Empresa que fabricó el artículo.

La reseña está delimitada con triples comillas invertidas. \
Formatee su respuesta como un objeto JSON con \
"Artículo" y "Marca" como claves.
Si la información no está presente, use "desconocido" \
como el valor.
Haga su respuesta lo más breve posible.
Texto de la reseña: '''{revision_lampara}'''
"""
response = get_completion(prompt)
print(response)


{
  "Artículo": "lámpara",
  "Marca": "Lumina"
}


## Hacer varias tareas a la vez

In [8]:
prompt = f"""Identifique los siguientes elementos del texto de revisión:
- Sentimiento (positivo o negativo)
- ¿El revisor está expresando enojo? (verdadero o falso)
- Artículo comprado por el revisor
- Empresa que fabricó el artículo.

La reseña está delimitada con triples comillas invertidas. \
Formatee su respuesta como un objeto JSON con \
"Sentimiento", "Ira", "Artículo" y "Marca" como claves.
Si la información no está presente, use "desconocido" \
como el valor.
Haga su respuesta lo más breve posible.
Formatee el valor de Ira como un valor booleano.
Texto de la reseña: '''{revision_lampara}'''
"""
response = get_completion(prompt)
print(response)



{
  "Sentimiento": "positivo",
  "Ira": false,
  "Artículo": "lámpara",
  "Marca": "Lumina"
}


## Inferir temas

In [9]:
historia = """
En una encuesta reciente realizada por el gobierno,
Se pidió a los empleados del sector público que calificaran su nivel
de satisfacción con el departamento en el que trabajan.
Los resultados revelaron que la NASA fue la más popular.
departamento con un índice de satisfacción del 95%.

Un empleado de la NASA, John Smith, comentó sobre los hallazgos,
diciendo: "No me sorprende que la NASA esté en la cima.
Es un gran lugar para trabajar con gente increíble y
increíbles oportunidades. Estoy orgulloso de ser parte de
una organización tan innovadora".

Los resultados también fueron bien recibidos por el equipo directivo de la NASA,
con el director Tom Johnson afirmando: "Estamos encantados de
escuchar que nuestros empleados están satisfechos con su trabajo en la NASA.
Contamos con un equipo talentoso y dedicado que trabaja incansablemente
para lograr nuestras metas, y es fantástico ver que sus
el trabajo duro está dando sus frutos".

La encuesta también reveló que la
La Administración del Seguro Social tuvo la satisfacción más baja
calificación, con solo el 45% de los empleados indicando que estaban
satisfecho con su trabajo. El gobierno se ha comprometido a
abordar las inquietudes planteadas por los empleados en la encuesta y
trabajar para mejorar la satisfacción laboral en todos los departamentos.
"""

In [16]:
prompt = f"""Determine cinco temas que se están discutiendo en el 
siguiente texto, que está delimitado por triples comillas simples.

Haga que cada tema tenga una o dos palabras.

Formatee su respuesta, sin enumerar, sino como una lista de elementos separados por comas (,)

Ejemplo de texto: '''{historia}'''
"""
response = get_completion(prompt)
print(response)

Departamentos públicos, Satisfacción laboral, NASA, Administración del Seguro Social, Encuesta gubernamental.


In [18]:
lista_temas = response.split(sep=',')

In [19]:
print(lista_temas)

['Departamentos públicos', ' Satisfacción laboral', ' NASA', ' Administración del Seguro Social', ' Encuesta gubernamental.']


## Hacer una alerta de noticias para ciertos temas



In [20]:
prompt = f"""Determine si cada elemento de la siguiente lista de 
temas es un tema en el texto a continuación, que
está delimitado con tres comillas graves.

Dé su respuesta como una lista con 0 o 1 para cada tema.
Lista de temas: {",".join(lista_temas)}
Ejemplo de texto: '''{historia}'''
"""
response = get_completion(prompt)
print(response)

Departamentos públicos: 1
Satisfacción laboral: 1
NASA: 1
Administración del Seguro Social: 1
Encuesta gubernamental: 1


In [22]:

topic_dict = {i.split(': ')[0]: int(i.split(': ')[1]) for i in response.split(sep='\n')}
if topic_dict['NASA'] == 1:
    print('ALERTA, nueva historia sobre NASA')

ALERTA, nueva historia sobre NASA
